## Import CSV

In [20]:
#read csv
import pandas as pd
features = pd.read_csv("train_set_ohe.csv")
features.head()

,Departure_FRA,Departure_STN,Departure_SXF,Destination_FRA,Destination_STN,Destination_SXF,Price_Dev_Cat_Falling,Price_Dev_Cat_Rising,Price_Dev_Cat_Steady,Request_Day,...,Same_Day_route_Flight,Is_Holiday_UK,Is_Holiday_GER,Is_School_Holiday_BER,Is_School_Holiday_FRA,Is_School_Holiday_UK,flight_unique_id_same_flight_date,Same_Day_Route_Flight_avg_price,Same_Day_Route_Flight_avg_price_utd,Same_Day_Request_route_Flight_price
0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,...,[],0,0,1,0,0,0,0.0,0.0,0.0
1,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,...,[],0,0,1,0,0,0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,5.0,...,[],0,0,1,0,0,0,0.0,0.0,0.0
3,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,5.0,...,[],0,0,1,0,0,0,0.0,0.0,0.0
4,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,...,[],0,0,1,0,0,0,0.0,0.0,0.0


## Step 1: Split into Predictors and Response Variable

In [22]:
features = features[["Departure_FRA", "Departure_STN", "Departure_SXF", "Destination_FRA", "Destination_STN", "Destination_SXF", "Price_Dev_Cat_Falling", 
                                  "Price_Dev_Cat_Rising", "Price_Dev_Cat_Steady", "Price_In_Eur",
                                  "Price_Dev", "Price_Dev_Three_Days", "Same_Day_Request_route_Flight_price",
                                  "Request_Month", "Request_Time", "Request_Day", "Flight_Day", "Departure_hour",
                                  "Hours_to_Flight", "Request_Count", "Request_Count_Sum", "Last_Request_Bool", "Is_Holiday_UK", "Is_Holiday_GER",
                                  "Is_School_Holiday_BER", "Is_School_Holiday_FRA", "Is_School_Holiday_UK", "buy"]]
features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83624 entries, 0 to 83623
Data columns (total 28 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   Departure_FRA                        83624 non-null  float64
 1   Departure_STN                        83624 non-null  float64
 2   Departure_SXF                        83624 non-null  float64
 3   Destination_FRA                      83624 non-null  float64
 4   Destination_STN                      83624 non-null  float64
 5   Destination_SXF                      83624 non-null  float64
 6   Price_Dev_Cat_Falling                83624 non-null  float64
 7   Price_Dev_Cat_Rising                 83624 non-null  float64
 8   Price_Dev_Cat_Steady                 83624 non-null  float64
 9   Price_In_Eur                         83624 non-null  float64
 10  Price_Dev                            83624 non-null  float64
 11  Price_Dev_Three_Days        

In [23]:
import numpy as np

# Labels are the values we want to predict
y = np.array(features['buy'])

# Remove labels from feature table
features = features.drop(['buy'], axis=1)

In [ ]:
features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83624 entries, 0 to 83623
Data columns (total 27 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   Departure_FRA                        83624 non-null  float64
 1   Departure_STN                        83624 non-null  float64
 2   Departure_SXF                        83624 non-null  float64
 3   Destination_FRA                      83624 non-null  float64
 4   Destination_STN                      83624 non-null  float64
 5   Destination_SXF                      83624 non-null  float64
 6   Price_Dev_Cat_Falling                83624 non-null  float64
 7   Price_Dev_Cat_Rising                 83624 non-null  float64
 8   Price_Dev_Cat_Steady                 83624 non-null  float64
 9   Price_In_Eur                         83624 non-null  float64
 10  Price_Dev                            83624 non-null  float64
 11  Price_Dev_Three_Days        

In [27]:
# Saving feature names for later use
feature_list = list(features.columns)

# Convert to numpy array
features = np.array(features)


## Step 2: Split into Train and Test Set

In [28]:
# Create Training and Test Sets
from sklearn.model_selection import train_test_split

trainX, testX, trainY, testY = train_test_split(features, y, test_size= 0.25, random_state=42, shuffle=True, stratify=y)

In [46]:
# reduced featured space to 18 with PCA
from sklearn.decomposition import PCA
pca2 = PCA(n_components=18)
pca2.fit(trainX)
trainX_reduced=pca2.transform(trainX)
testX_reduced=pca2.transform(testX)
print(trainX_reduced.shape)
print(testX_reduced.shape)

(62718, 18)
(20906, 18)


In [40]:
X_reduced

array([[-5.77088590e+01, -2.93707619e+01, -6.89128460e+01, ...,
         1.13303939e-01,  8.29910283e-02,  8.93284232e-01],
       [ 1.08709299e+03,  1.47711979e+01,  1.39314250e+01, ...,
         9.09627547e-04, -9.40555897e-03, -1.39826240e-01],
       [ 8.55478871e+02,  1.16037172e+01,  2.86838959e+01, ...,
         1.36711786e-01, -4.73356148e-02,  4.33834216e-01],
       ...,
       [-5.95524771e+02, -7.00746331e+01,  7.82423556e+01, ...,
         1.28955625e-01, -4.05944653e-02,  7.90860966e-02],
       [-6.44164228e+02, -5.63800988e+01, -6.48335593e+01, ...,
         7.37661840e-01, -4.52556517e-01,  3.36892490e-01],
       [ 6.51004151e+01, -9.41830928e+00, -6.54052043e+00, ...,
         8.13985573e-01, -5.26468821e-01, -2.17274553e-01]])

In [41]:
trainX

array([[0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 1.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 1., ..., 1., 0., 0.],
       [0., 1., 0., ..., 0., 0., 1.],
       [0., 1., 0., ..., 0., 1., 1.]])

In [48]:
print('Training Features Shape:', trainX.shape)
print('Training Labels Shape:', trainY.shape)
print('Testing Features Shape:', testX.shape)
print('Testing Labels Shape:', testY.shape)

Training Features Shape: (62718, 27)
Training Labels Shape: (62718,)
Testing Features Shape: (20906, 27)
Testing Labels Shape: (20906,)


## Step 3: Fit Model with Training Data

In [ ]:
# Import the model we are using
from sklearn.ensemble import RandomForestClassifier
# Instantiate model with 1000 decision trees
rfc = RandomForestClassifier(n_estimators = 1000, random_state = 42)

In [ ]:
# Train the model on training data
rfc.fit(trainX, trainY)
rfc.score(testX, testY)

0.8965368793647757

In [ ]:
# Use the forest's predict method on the test data
predictions = rfc.predict(testX)

# Calculate the absolute errors
errors = abs(predictions - testY)

from sklearn.metrics import accuracy_score

accuracy_score(testY, predictions)

## Step 4: Hyperparameter Tuning

In [ ]:
from pprint import pprint

print('Parameters currently in use:\n')
pprint(rfc.get_params())

Parameters currently in use:

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 1000,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 42,
 'verbose': 0,
 'warm_start': False}


In [ ]:
from sklearn.model_selection import RandomizedSearchCV

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

pprint(random_grid)

{'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}


In [ ]:
rfc = RandomForestClassifier()

rfc_random = RandomizedSearchCV(estimator=rfc, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = 1)

rfc_random.fit(trainX, trainY)

Fitting 3 folds for each of 100 candidates, totalling 300 fits
[CV] END bootstrap=True, max_depth=30, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=400; total time=  26.8s
[CV] END bootstrap=True, max_depth=30, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=400; total time=  26.9s
[CV] END bootstrap=True, max_depth=30, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=400; total time=  26.9s
[CV] END bootstrap=True, max_depth=10, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=2000; total time= 1.4min
[CV] END bootstrap=True, max_depth=10, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=2000; total time= 1.4min
[CV] END bootstrap=True, max_depth=10, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=2000; total time= 1.4min
[CV] END bootstrap=False, max_depth=10, max_features=sqrt, min_samples_leaf=2, min_samples_split=5, n_estimators=1200;

RandomizedSearchCV(cv=3, estimator=RandomForestClassifier(), n_iter=100,
                   n_jobs=1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, 110,
                                                      None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000, 1200, 1400, 1600,
                                                         1800, 2000]},
                   random_state=42, verbose=2)

In [ ]:
rfc_random.best_params_


{'bootstrap': False,
 'max_depth': 50,
 'max_features': 'auto',
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 1000}

In [ ]:
rfc_random.cv_results_

{'mean_fit_time': array([ 24.82736818,  78.589475  ,  64.46673306, 172.65908583,
         62.85650675,  66.70750356,  88.86899598,  54.08206256,
         92.1267705 ,  96.75259884,  22.81194258,  73.05169996,
        106.18610128,  84.82907391,  68.27230128, 151.59677259,
         52.75051252,  61.25120656, 154.27152276,  36.57026656,
        160.09648919, 144.50635457,  79.65210907, 110.71486886,
        125.82838972,  36.91304954, 131.09634948,  58.65374859,
        165.56900628,  70.29079366,  96.97087399,  70.61245449,
         72.62730797,  45.83159773, 105.14994899,  66.0689559 ,
         70.34583147,  50.64591567,  47.21063368,  61.33855192,
          7.86721381,  34.93774772,  45.54879411,  22.28815913,
         17.28596775,  63.35140149, 123.59110665,  32.69841838,
         99.43994228,  52.33062331, 157.02055248, 170.70942593,
         82.1840016 ,  82.14970954, 123.06727823,  49.85850994,
         61.82590652, 109.52588646,  87.82350294,  79.52571313,
         12.49682728,  

In [ ]:
rfc_random.best_estimator_

RandomForestClassifier(bootstrap=False, max_depth=50, n_estimators=1000)

In [ ]:
# Import the model we are using
from sklearn.ensemble import RandomForestClassifier
# Instantiate model with 1000 decision trees
rfc = RandomForestClassifier(n_estimators = 1000, max_depth=50, bootstrap=False, max_features='auto', min_samples_leaf=1, min_samples_split=2)
# rfc = RandomForestClassifier(bootstrap=True, max_depth=80, max_features='auto', min_samples_leaf=1, min_samples_split=5, n_estimators=600)

In [47]:
# Train the model on training data
rfc.fit(trainX_reduced, trainY)
rfc.score(testX_reduced, testY)

0.8949583851525877

In [49]:
rfc.fit(trainX, trainY)
rfc.score(testX, testY)

0.90438151726777